### How to load yaml cam configs and access info

- supply one config function to access cam dicts for noise properties
- access functions in cam modules to access cam noise properties

#### cam dict loading

- keep it simple: load yamls in sequence from most general to most user specific directory
- while walking through those directories construct a master dict that just gets updated with new info and would thus overwrite existing entries with the most user specific ones taking precendence

Something like:

```
def config.loadCamInfo():
    camdict = {}
    for dir in config_cam_dirs:
        for cam_yaml in find_yamls(dir): # perhaps we allow several files that get merged but not necessary
            camdict.update(yaml.safe_load(cam_yaml)
    return camdict
```

#### access functions

- camera access functions that consult both hardcoded info and config cam properties to retrieve required info (precedence is config file over hardcoded)
- examples below in fakecam classes


### How to install site wide config files

Here I suggest adopting a simple approach as we do for the plugin packages. A site-config repo would contain the cam config files (yaml) and an `install_cams.py` script that copies the relevant camera file(s) to the desired level config directory. (very much similar to the `install_plugins.py` script)

If we have just one camera config file, rather than a camera directory, there may have to be a way to merge entries into the current directory.

### mock functionality that would go in some fashion into PYME.config

In [1]:
# this code is supposed to simulate functionality that will reside in the PYME.config module eventually
# it glosses obver the details how the directories are traversed etc but gives the general idea below

# this will be filled by a cached read that gets triggered the first time getCamNoiseProps is used
_camNoiseProps = None

# here we read the two files we made (for Andor emCCD and sCMOS) from the info in the hardcoded dicts
# in reality we would walk through the config directories
# this should be fairly trivial to implement and just needs agreement how we do this, i.e. one single file
# per config directory or allow several files with a 'cameras' subdirectory in each config directory
import yaml
import glob
def config_readCamNoiseProps():
    global _camNoiseProps
    _camNoiseProps = {}
    for yamlfile in glob.glob('*.yaml'):
        with open(yamlfile,'r') as fi:
            _camNoiseProps.update(yaml.safe_load(fi))

            
def config_getCamNoiseProps(serno,preampmode):
    if _camNoiseProps is None:
        config_readCamNoiseProps()
    if serno in _camNoiseProps:
        try:
            return _camNoiseProps[serno]['noiseProperties'][preampmode]
        except KeyError:
            return None
    else:
        return None

### Heterogeneous dictionary structure that is adopted

- each camera has an entry (dict-like) accessed by its serial number
- the critical stuff is in the `noiseProperties` slot (can discuss what naming convention to use, CapWords vs underscore separated, not sure what the Python conventions are nor if PYME religously follows those)
- the `noiseProperties` slot has, based an camera type, sub entries listed by a (generally string based) pre-amp specification 
- by convention a `model` slot is present to help identify cam type for the reader; currently it is not checked so no stringent model naming is required
- other slots can be added, e.g. a `comment` slot if so desired, for now they are completely ignored

Example of sCMOS type entry:

In [2]:
print(yaml.dump(config_getCamNoiseProps('VSC-00954','12-bit (low noise)'), default_flow_style=False, default_style='' ))

ADOffset: 100
ElectronsPerCount: 0.28
ReadNoise: 1.1
SaturationThreshold: 2047



Example of emCCD type entry:

In [3]:
print(yaml.dump(config_getCamNoiseProps(1823,'EM preamp mode 0'), default_flow_style=False, default_style='' ))

ADOffset: 971
DefaultEMGain: 150
ElectronsPerCount: 27.32
NGainStages: 536
ReadNoise: 109.8
SaturationThreshold: 16383



### A couple of fake cams to exercise the cam property lookup

The `@property` bits suggest possible implementations of the lookup mechanism

In [4]:
class fakecamBase(object):
    _hardcoded_noise_properties = {} # no hardcoded cams
    
    def __init__(self):
        pass
    
    def _preamp_mode(self):
        """
        Should return a representation of the camera preamp mode that can be used as a dictionary key.
        """        
        return 'default'
        
    @property
    def noise_properties(self):
        """return the noise properties for the given camera

        """
        serno = self.GetSerialNumber()
        np = config_getCamNoiseProps(serno,self._preamp_mode())
        if np is not None:
            return np
        # if we get to here fall back to hardcoded cameras
        try:
            return self._hardcoded_noise_properties[serno]['noiseProperties'][self._preamp_mode()]
        except KeyError: # last resort is default set of values
            raise RuntimeError('camera specific noise props not found for serial no "%s" and preamp mode "%s"' 
                               % (serno,self._preamp_mode()))

In [5]:
class fakecamCMOS(fakecamBase):   
    
    def __init__(self):
        self.serno = 'CSC-00425'
        pass
    
    def GetSerialNumber(self):
        return self.serno
    
    def SetSerialNumber(self, serno):
        self.serno = serno
    
    def GetSimpleGainMode(self):
        return '12-bit (low noise)'

    def _preamp_mode(self):
        return self.GetSimpleGainMode()

In [6]:
fc = fakecamCMOS()

In [7]:
config_getCamNoiseProps(fc.GetSerialNumber(),fc._preamp_mode())

{'ADOffset': 100,
 'ElectronsPerCount': 0.45,
 'ReadNoise': 1.21,
 'SaturationThreshold': 1776}

In [8]:
fc.noise_properties

{'ADOffset': 100,
 'ElectronsPerCount': 0.45,
 'ReadNoise': 1.21,
 'SaturationThreshold': 1776}

In [9]:
def getpreamp(padict,serial):
    if serial in padict:
        return padict[serial]
    else:
        return 2

# this function would do some invertable way of creating a unique pream setting identifier
# so this likely *not* the eventual way that will be chosen
def getPreampModeString(mode):
    return 'EM preamp mode %d' % mode


class fakecamEmCCD(fakecamBase):
    _noise_properties = {
        4532 : { #Gain setting of 1
        'ReadNoise' : 56.1,
        'ElectronsPerCount' : 3.6,
        'NGainStages' : 345,
        'ADOffset' : 203,
        'DefaultEMGain' : 90,
        'SaturationThreshold' : 5.4e4#(2**16 -1)
        }
    } # no hardcoded cams
    _preamp_gains = {
        4532 : 1
    }
        
    def __init__(self):
        self._wrapOldStyleNPs()
        self.serno = 1823
        self.preampGain = 0
    
    # instead of manually rewriting the old style noise props we provide code to convert old entries to the new style
    def _wrapOldStyleNPs(self):
        configdict = {}
        for serial in self._noise_properties:
            camdict = {}
            camdict['noiseProperties'] = { getPreampModeString(self._preamp_gains[serial]) : self._noise_properties[serial]}
            camdict['model'] = 'Andor IXON emCCD' # could these be different IXON models?
            configdict[serial] = camdict
        self._hardcoded_noise_properties = configdict

    def GetSerialNumber(self):
        return self.serno
    
    def SetSerialNumber(self, serno):
        self.serno = serno
    
    # we are overriding the base class implementation to supply the preamp info
    # for the actual IXONs a more complex conversion function may be required if there is more than one gain
    # to encode in this field
    def _preamp_mode(self):
        return 'EM preamp mode %d' % self.preampGain

In [10]:
fce = fakecamEmCCD()

In [11]:
fce.SetSerialNumber(5414)
print(fce.preampGain)
fce.noise_properties

0


{'ADOffset': 413,
 'DefaultEMGain': 90,
 'ElectronsPerCount': 25.24,
 'NGainStages': 536,
 'ReadNoise': 61.33,
 'SaturationThreshold': 16383}

In [12]:
# now access the hardcoded cam
fce.SetSerialNumber(4532)
fce.preampGain = 1
print(fce.preampGain)
fce.noise_properties

1


{'ADOffset': 203,
 'DefaultEMGain': 90,
 'ElectronsPerCount': 3.6,
 'NGainStages': 345,
 'ReadNoise': 56.1,
 'SaturationThreshold': 54000.0}

In [13]:
# now access the hardcoded cam
fce.SetSerialNumber(4532)
fce.preampGain = 0
print(fce.preampGain)
fce.noise_properties

0


RuntimeError: camera specific noise props not found for serial no "4532" and preamp mode "EM preamp mode 0"